In [1]:
import time
import datetime
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

In [2]:
brave_ops = Options()
brave_ops.binary_location="/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
brave = Chrome(executable_path="/Users/clintonricksiii/documents/armorcritical/my_development/refbot/chromedriver_v80", options=brave_ops)


In [3]:
"""
TEST AREA

-Need to format data types after obtained from cta site
-Create separate player table (dataframe) to contain player aliases
"""
pd.set_option('display.max_columns', None)


In [4]:
games_df = pd.DataFrame(columns=['Number of Teams',
 'Recharge Rate',
 'Game ID',
 'Laser Damage',
 'Special Damage',
 'Map',
 'Game Mode',
 'Date',
 'Player Count',
 'Referee'])

In [5]:
stats_df = pd.DataFrame(columns=["Game ID",
                    "Player",
                    "Outcome",
                    "Team",
                    "Kills",
                    "Deaths",
                    "+/-",
                    "Flag Captures",
                    "Flag Attempts",
                    "Switch",
                    "Elected",
                    "Assassinations",
                    "Score",
                    "Time Played",
                    "Death Chart",
                    "Damage Dealt",
                    "Damage Taken",
                    "DR",
                    "DKR",
                    "DDF",
                    "Laser DD",
                    "Laser DT",
                    "Missile DD",
                    "Missile DT",
                    "Grenade DD",
                    "Grenade DT",
                    "Bouncy DD",
                    "Bouncy DT",
                    "Killed Most - Players",
                    "Killed Most - Values",
                    "Killed Most By - Players",
                    "Killed Most By - Values",
                    "Damaged Most - Players",
                    "Damaged Most - Values",
                    "Damaged Most By - Players",
                    "Damaged Most By - Values"])

In [10]:
def add_to_dataframes(game_ids, games_df, stats_df):
    
    for game_id in game_ids:
    
        match_entry = {"Game ID" : game_id,
                        "Date": 0,
                        "Referee": 0,
                        "Number of Teams": 0,
                        "Player Count": 0,
                        "Map": 0,
                        "Game Mode": 0,
                        "Laser Damage": 0,
                        "Special Damage": 0,
                        "Recharge Rate": 0}
        
        brave.get("http://cta.critical-hq.net/game.php?matchID=" + str(game_id))
        
        if brave.find_elements_by_tag_name("p")[0].text == "Game was not found":
            continue
        else:
            for k in match_entry.keys():
                if k == "Date":
                    match_entry[k] = datetime.datetime.strptime(brave.find_element_by_xpath('//*[@title="{}"]'.format(k)).text.split("\n")[-1], "%m.%d.%Y %I:%M %p")
                elif k == "Game ID" or k == "Number of Teams" or k == "Player Count":
                    try:
                        match_entry[k] = int(brave.find_element_by_xpath('//*[@title="{}"]'.format(k)).text.split("\n")[-1])
                    except:
                        match_entry[k] = float("NaN")
                else:    
                    match_entry[k] = brave.find_element_by_xpath('//*[@title="{}"]'.format(k)).text.split("\n")[-1]

            games_df = games_df.append(match_entry, ignore_index=True)

            stats_entry = {"Game ID" : match_entry["Game ID"],
                            "Player" : float("NaN"),
                            "Outcome": float("NaN"),
                            "Team": float("NaN"),
                            "Kills": float("NaN"),
                            "Deaths": float("NaN"),
                            "+/-": float("NaN"),
                            "Flag Captures": float("NaN"),
                            "Flag Attempts": float("NaN"),
                            "Switch": float("NaN"),
                            "Elected": float("NaN"),
                            "Assassinations": float("NaN"),
                            "Score": float("NaN"),
                            "Time Played": float("NaN"),
                            "Death Chart": float("NaN"),
                            "Damage Dealt": float("NaN"),
                            "Damage Taken": float("NaN"),
                            "DR": float("NaN"),
                            "DKR" : float("NaN"),
                            "DDF": float("NaN"),
                            "Laser DD": float("NaN"),
                            "Laser DT": float("NaN"),
                            "Missile DD" : float("NaN"),
                            "Missile DT": float("NaN"),
                            "Grenade DD": float("NaN"),
                            "Grenade DT": float("NaN"),
                            "Bouncy DD": float("NaN"),
                            "Bouncy DT": float("NaN"),
                            "Killed Most - Players": float("NaN"),
                            "Killed Most - Values": float("NaN"),
                            "Killed Most By - Players": float("NaN"),
                            "Killed Most By - Values": float("NaN"),
                            "Damaged Most - Players": float("NaN"),
                            "Damaged Most - Values": float("NaN"),
                            "Damaged Most By - Players": float("NaN"),
                            "Damaged Most By - Values": float("NaN")
                            }

            basic_df = pd.read_html(brave.find_element_by_id("basic").get_attribute('outerHTML'))
            damage_df = pd.read_html(brave.find_element_by_id("damage").get_attribute('outerHTML'))
            weapon_df = pd.read_html(brave.find_element_by_id("weapon").get_attribute('outerHTML'))

            player_list = []

            for table in basic_df:
                for player in table.iloc[:, 1]:
                    if player != "Totals":
                        player_list.append(player)
            
            for player in player_list:
                player_entry = stats_entry
                for table in basic_df:
                    if player in list(table.iloc[:, 1]):
                        player_entry["Player"] = player

                        if len(table.iloc[:, 1].name.split(" ")) == 3:
                            player_entry["Outcome"] = 1
                        else:
                            player_entry["Outcome"] = 0

                        player_entry["Team"] = int(table.iloc[:, 1].name.split(" ")[1])
                        player_entry["Kills"] = int(table[table.iloc[:, 1] == player]["K"].values[0])
                        player_entry["Deaths"] = int(table[table.iloc[:, 1] == player]["D"].values[0])
                        player_entry["+/-"] = int(table[table.iloc[:, 1] == player]["+/-"].values[0])

                        if "Flag" in table.columns:
                            player_entry["Flag Captures"] = int(table[table.iloc[:, 1] == player]["Flag"].values[0].split("/")[0])
                            player_entry["Flag Attempts"] = int(table[table.iloc[:, 1] == player]["Flag"].values[0].split("/")[1])

                        if "Switch" in table.columns:
                            player_entry["Switch"] = int(table[table.iloc[:, 1] == player]["Switch"].values[0])

                        if "Elected" in table.columns:
                            player_entry["Elected"] = int(table[table.iloc[:, 1] == player]["Elected"])

                        if "Assn" in table.columns:
                            player_entry["Assassinations"] = int(table[table.iloc[:, 1] == player]["Assn"].values[0])

                        player_entry["Score"] = int(table[table.iloc[:, 1] == player]["Score"].values[0])
                        player_entry["Time Played"] = int(table[table.iloc[:, 1] == player]["TP"].values[0].split("m")[0])
                        player_entry["Death Chart"] = "http://cta.critical-hq.net/chart.php?player=" + player + "&matchID=" + str(player_entry["Game ID"])

                for table in damage_df:
                    if player in list(table.iloc[:, 1]):
                        player_entry["Damage Dealt"] = int(table[table.iloc[:, 1] == player]["DD"].values[0])
                        player_entry["Damage Taken"] = int(table[table.iloc[:, 1] == player]["DT"].values[0])
                        
                        try:
                            player_entry["DR"] = player_entry["Damage Dealt"]/player_entry["Damage Taken"]
                        except:
                            player_entry["DR"] = int(table[table.iloc[:, 1] == player]["DR"].values[0])
                            
                        player_entry["DKR"] = int(table[table.iloc[:, 1] == player]["DKR"].values[0])
                        player_entry["DDF"] = int(table[table.iloc[:, 1] == player]["DDF"].values[0])

                for table in weapon_df:
                    if player in list(table.iloc[:, 1]):
                        player_entry["Laser DD"] = int(table[table.iloc[:, 1] == player]["Laser"].values[0].split(" / ")[0])
                        player_entry["Laser DT"] = int(table[table.iloc[:, 1] == player]["Laser"].values[0].split(" / ")[1])
                        player_entry["Missile DD"] = int(table[table.iloc[:, 1] == player]["Missile"].values[0].split(" / ")[0])
                        player_entry["Missile DT"] = int(table[table.iloc[:, 1] == player]["Missile"].values[0].split(" / ")[1])
                        player_entry["Grenade DD"] = int(table[table.iloc[:, 1] == player]["Grenade"].values[0].split(" / ")[0])
                        player_entry["Grenade DT"] = int(table[table.iloc[:, 1] == player]["Grenade"].values[0].split(" / ")[1])
                        player_entry["Bouncy DD"] = int(table[table.iloc[:, 1] == player]["Bouncy"].values[0].split(" / ")[0])
                        player_entry["Bouncy DT"] = int(table[table.iloc[:, 1] == player]["Bouncy"].values[0].split(" / ")[1])

                brave.get("http://cta.critical-hq.net/expand.php?view=Killed&matchID=" + str(player_entry["Game ID"]) + "&player=" + player)
                names = []
                amounts = []

                for item in brave.find_elements_by_xpath('//*[@style="font-size:12px;"]'):
                    names.append(item.text.split(" ")[0])
                    amounts.append(item.text.split(" ")[1][1:-1])

                player_entry["Killed Most - Players"] = " ".join(names)
                player_entry["Killed Most - Values"] = " ".join(amounts)

                brave.get("http://cta.critical-hq.net/expand.php?view=KilledBy&matchID=" + str(player_entry["Game ID"]) + "&player=" + player)
                names = []
                amounts = []

                for item in brave.find_elements_by_xpath('//*[@style="font-size:12px;"]'):
                    names.append(item.text.split(" ")[0])
                    amounts.append(item.text.split(" ")[1][1:-1])

                player_entry["Killed Most By - Players"] = " ".join(names)
                player_entry["Killed Most By - Values"] = " ".join(amounts)

                brave.get("http://cta.critical-hq.net/expand.php?view=Damaged&matchID=" + str(player_entry["Game ID"]) + "&player=" + player)
                names = []
                amounts = []

                for item in brave.find_elements_by_xpath('//*[@style="font-size:12px;"]'):
                    names.append(item.text.split(" ")[0])
                    amounts.append(item.text.split(" ")[1][1:-1])

                player_entry["Damaged Most - Players"] = " ".join(names)
                player_entry["Damaged Most - Values"] = " ".join(amounts)

                brave.get("http://cta.critical-hq.net/expand.php?view=DamagedBy&matchID=" + str(player_entry["Game ID"]) + "&player=" + player)
                names = []
                amounts = []

                for item in brave.find_elements_by_xpath('//*[@style="font-size:12px;"]'):
                    names.append(item.text.split(" ")[0])
                    amounts.append(item.text.split(" ")[1][1:-1])

                player_entry["Damaged Most By - Players"] = " ".join(names)
                player_entry["Damaged Most By - Values"] = " ".join(amounts)
                                
                stats_df = stats_df.append(player_entry, ignore_index=True)
        
        games_df.to_csv("games_df.csv", encoding='utf-8')
        stats_df.to_csv("stats_df.csv", encoding="utf-8")
        print("Completed GameID: ", game_id)
    
    return games_df, stats_df        




In [11]:
games_df = pd.read_csv("games_df.csv")
games_df = games_df.iloc[:, 1:]

In [12]:
stats_df = pd.read_csv("stats_df.csv")
stats_df = stats_df.iloc[:, 1:]

In [13]:
games_df, stats_df = add_to_dataframes([game for game in range(130, 24191)], games_df, stats_df)

Completed GameID:  130
Completed GameID:  131
Completed GameID:  132
Completed GameID:  133
Completed GameID:  134
Completed GameID:  136
Completed GameID:  137
Completed GameID:  138
Completed GameID:  139
Completed GameID:  140
Completed GameID:  141
Completed GameID:  142
Completed GameID:  143
Completed GameID:  144
Completed GameID:  145
Completed GameID:  146
Completed GameID:  147
Completed GameID:  148
Completed GameID:  150
Completed GameID:  151
Completed GameID:  152
Completed GameID:  153
Completed GameID:  154
Completed GameID:  155
Completed GameID:  156
Completed GameID:  157
Completed GameID:  158
Completed GameID:  159
Completed GameID:  160
Completed GameID:  161
Completed GameID:  162
Completed GameID:  163
Completed GameID:  164
Completed GameID:  165
Completed GameID:  166
Completed GameID:  167
Completed GameID:  168
Completed GameID:  169
Completed GameID:  170
Completed GameID:  171
Completed GameID:  172
Completed GameID:  173
Completed GameID:  174
Completed G

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=81.0.4044.138)
